In [1]:
import requests
from bs4 import BeautifulSoup, Comment
from pathlib import Path
import os
import os.path
from os import path

In [2]:
url = 'http://sibgrapi.sid.inpe.br/col/sid.inpe.br/banon/2001/03.30.15.38.24/doc/mirror.cgi/About?languagebutton=en&languagerep1=sid.inpe.br/banon/2001/05.04.15.05&languagerep2=dpi.inpe.br/banon/1999/05.03.22.11&firstlanguagerep=sid.inpe.br/banon/2001/05.04.15.05&submissionformrep=dpi.inpe.br/banon-pc2@80/2006/08.30.19.42&returnbutton=yes'
req_main_page = requests.get(url)

In [3]:
main_page = BeautifulSoup(req_main_page.content, 'html.parser')
table = main_page.find_all('table')

In [4]:
def getYearConf(string):
    counter = 0
    year = ""
    for char in string:
        if char.isdigit():
            year += char
            counter += 1
        if(counter == 4):
            return year

In [5]:
years = []
all_links = []
for tag_a in table[0].find_all('a'):
    year = getYearConf(tag_a.text)
    link = tag_a.get('href')
    dois_pontos = ":"
    format = dois_pontos + str(year) + dois_pontos
    if year not in years and year != None:
        years.append(year)
        all_links.append(link)
    elif year in years and year != None:
        if year == '2021':
            length = len(all_links)
            all_links[length - 1] = link

In [6]:
confs = []
for i in range(len(years)):
    lista = [int(years[i]), all_links[i]]
    confs.append(lista)

In [7]:
main_pages_papers_confs = []
for conf in confs:
    req_main_page_paper = requests.get(conf[1])
    main_page_paper = BeautifulSoup(req_main_page_paper.content, 'html.parser')
    frame = main_page_paper.find_all('frame')
    yearLinks = [conf[0], frame[1].get('src')]
    main_pages_papers_confs.append(yearLinks)

In [8]:
def getTagsAfterComments(link):
    req_main_page_paper = requests.get(link)
    main_page_paper = BeautifulSoup(req_main_page_paper.content, 'html.parser')
    separator = "aaaaaaaaaaaa"
    comment = main_page_paper.find(string=lambda text: isinstance(text, Comment) and separator in text)
    if comment:
        next_element = comment.find_next_siblings()
        if next_element:
            return next_element
        else:
            return None
                    

In [12]:
def getQtPapersCategory(html):
    countersCategory = {}
    key = ""
    counterPapers = 0
    for tag in html:
        if tag.get('class') and "titleAuthorTABLE" in tag.get('class'):
            counterPapers = counterPapers + 1
        if tag.name == "b":
            if counterPapers == 0:
                key = tag.text
            countersCategory[key] = counterPapers
            counterPapers = 0
            key = tag.text
    
    countersCategory[key] = counterPapers
    return countersCategory

In [23]:
def getLinksPapers(qt, html, ano):
    links_papers = {}
    counter = 0
    all_links = []
    key_html = ""
    for tag in html:
        if tag.get('class') and "titleAuthorTABLE" in tag.get('class'):
            all_links.append(tag.find('a').get('href'))
            counter = counter + 1
        if tag.name == "b":
            if counter == 0:
                key_html = tag.text
            if qt[key_html] == counter:
                links_papers[key_html] = all_links
                all_links = []
                counter = 0
            key_html = tag.text
     
    links_papers[key_html] = all_links
    for key, links in links_papers.items():
        if not "Video" in key:
            for url in links:
                url = url.split("?")
                url = url[0][5:]
                url_formatted = "http://sibgrapi.sid.inpe.br/col/" + url + "/doc/thisInformationItemHomePage.html"
                req_paper_page = requests.get(url_formatted)
                paper_page = BeautifulSoup(req_paper_page.content, 'html.parser')
                frames = paper_page.find_all("frame")
                if len(frames) > 0:
                    req_paper_PDF = requests.get(frames[1].get('src'))
                    if "@80" in url:
                        filename_formatted = str(ano) + "/" + key + "-" + url[29:39] + ".pdf"
                    elif "lotufo" in url:
                        filename_formatted = str(ano) + "/" + key + "-" + url[31:41] + ".pdf"
                    else:
                        filename_formatted = str(ano) + "/" + key + "-" + url[26:37] + ".pdf"
                    filename = Path(filename_formatted)
                    filename.write_bytes(req_paper_PDF.content)
        


In [24]:
for lista in main_pages_papers_confs:
    print("Inicia processamento do ano", lista[0])
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    folder = "/home/priangel/Documentos/TCC/analysis-sibgrapi-papers/" + str(lista[0])
    if not path.exists(folder):
        os.mkdir(folder)
        html_links_paper = getTagsAfterComments(lista[1])
        if html_links_paper:
            qt_papers_category = getQtPapersCategory(html_links_paper)
            getLinksPapers(qt_papers_category, html_links_paper, lista[0])
        else:
            print("No links to paper")
    else:
        print(lista[0], "já processado")
        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

Inicia processamento do ano 2022
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2022 já processado
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Inicia processamento do ano 2021
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2021 já processado
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Inicia processamento do ano 2020
-----------------------------------------------------------------------------------------------------------------------------------